# **Fabric Lakehouse Consolidation Tool**


###### This notebook provides an automated solution for creating and managing consolidated lakehouses in Microsoft Fabric.
###### It allows you to create a unified lakehouse with shortcuts to multiple source lakehouses, organized by schemas, with intelligent filtering and refresh capabilities.

--- 
## What this tool does

###### This tool automates the creation of a **consolidated lakehouse** that acts as a centralized view of multiple source lakehouses (e.g., Bronze, Silver, Gold layers in a medallion architecture). Instead of copying data, it creates **shortcuts** (symbolic links) to tables in source lakehouses, organizing them into schemas for easy navigation.

---

# Use the tool

## Prerequisites

In [ ]:
#### PACKAGES

import requests
import json
from notebookutils import mssparkutils
import time
from typing import List, Dict, Optional

StatementMeta(, 1bfba26f-2861-45c5-b30b-1a3ba2850f1c, 4, Finished, Available, Finished)

In [ ]:
#### TOKEN GENERATION
token = mssparkutils.credentials.getToken("pbi")

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

base_url = "https://api.fabric.microsoft.com/v1"


## Functions

In [ ]:
def create_lakehouse(workspace_id: str, lakehouse_name: str) -> Optional[str]:
    """Creates a new lakehouse"""
    url = f"{base_url}/workspaces/{workspace_id}/lakehouses"
    payload = {
        "displayName": lakehouse_name
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code in [200, 201]:
        return response.json()["id"]
    else:
        print(f"Error creating lakehouse: {response.status_code} - {response.text}")
        return None

def get_lakehouse_tables(workspace_id: str, lakehouse_id: str) -> List[Dict]:
    """Gets all tables from a lakehouse"""
    url = f"{base_url}/workspaces/{workspace_id}/lakehouses/{lakehouse_id}/tables"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print(f"Error getting tables: {response.status_code} - {response.text}")
        return []

def get_existing_shortcuts(workspace_id: str, lakehouse_id: str, schema_name: str) -> List[str]:
    """Gets existing shortcuts in a specific schema"""
    url = f"{base_url}/workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts"
    response = requests.get(url, headers=headers)
    
    existing_shortcuts = []
    if response.status_code == 200:
        shortcuts = response.json().get("value", [])
        # Filter shortcuts that belong to this schema
        for shortcut in shortcuts:
            if shortcut.get("path", "").startswith(f"Tables/{schema_name}/"):
                existing_shortcuts.append(shortcut.get("name"))
    
    return existing_shortcuts

def delete_shortcut(workspace_id: str, lakehouse_id: str, shortcut_path: str, shortcut_name: str) -> bool:
    """Deletes a shortcut"""
    url = f"{base_url}/workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts/{shortcut_path}/{shortcut_name}"
    response = requests.delete(url, headers=headers)
    return response.status_code in [200, 204]

def create_shortcut(target_workspace_id: str, target_lakehouse_id: str, shortcut_path: str, 
                   shortcut_name: str, source_workspace_id: str, source_lakehouse_id: str, 
                   source_path: str) -> tuple[bool, str]:
    """Creates a shortcut according to Microsoft official documentation"""
    url = f"{base_url}/workspaces/{target_workspace_id}/items/{target_lakehouse_id}/shortcuts"
    
    payload = {
        "path": shortcut_path,
        "name": shortcut_name,
        "target": {
            "oneLake": {
                "workspaceId": source_workspace_id,
                "itemId": source_lakehouse_id,
                "path": source_path
            }
        }
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code in [200, 201]:
        return True, "Success"
    else:
        return False, f"{response.status_code} - {response.text}"

def verify_lakehouse(workspace_id: str, lakehouse_id: str) -> bool:
    """Verifies that a lakehouse exists and is accessible"""
    url = f"{base_url}/workspaces/{workspace_id}/lakehouses/{lakehouse_id}"
    response = requests.get(url, headers=headers)
    return response.status_code == 200

def filter_tables(tables: List[Dict], table_filter: List[str]) -> List[Dict]:
    """Filters tables based on the provided filter list"""
    if not table_filter:  # Empty list means all tables
        return tables
    
    # Filter tables that match the filter list (case-insensitive)
    filter_lower = [t.lower() for t in table_filter]
    filtered = [table for table in tables if table["name"].lower() in filter_lower]
    
    return filtered

def process_source(target_workspace_id: str, target_lakehouse_id: str, 
                  source_name: str, source_config: Dict, 
                  refresh_mode: bool = False) -> Dict:
    """
    Processes a single source to create shortcuts
    
    Args:
        refresh_mode: If True, only creates shortcuts for new tables
    """
    schema_name = source_config["schema_name"]
    source_workspace_id = source_config["workspace_id"]
    source_lakehouse_id = source_config["lakehouse_id"]
    table_filter = source_config.get("table_filter", [])
    
    print(f"\n{'='*70}")
    print(f"[SOURCE] {source_name.upper()}")
    print(f"{'='*70}")
    print(f"  Workspace ID: {source_workspace_id}")
    print(f"  Lakehouse ID: {source_lakehouse_id}")
    print(f"  Target Schema: {schema_name}")
    if table_filter:
        print(f"  Table Filter: {', '.join(table_filter)}")
    else:
        print(f"  Table Filter: All tables")
    print(f"  Mode: {'REFRESH' if refresh_mode else 'INITIAL SETUP'}")
    
    # Verify access to source lakehouse
    print(f"\n  🔍 Verifying access to source lakehouse...")
    if not verify_lakehouse(source_workspace_id, source_lakehouse_id):
        print(f"  ❌ Cannot access lakehouse {source_name}")
        print(f"     Check permissions and verify the ID is correct")
        return {"total": 0, "success": 0, "failed": 0, "skipped": 0}
    print(f"  ✅ Lakehouse accessible")
    
    # Get existing shortcuts if in refresh mode
    existing_shortcuts = []
    if refresh_mode:
        print(f"\n  📋 Getting existing shortcuts...")
        existing_shortcuts = get_existing_shortcuts(target_workspace_id, target_lakehouse_id, schema_name)
        print(f"  ℹ️  Found {len(existing_shortcuts)} existing shortcuts")
    
    # Get tables
    print(f"\n  📋 Getting table list...")
    all_tables = get_lakehouse_tables(source_workspace_id, source_lakehouse_id)
    
    if not all_tables:
        print(f"  ⚠️  No tables found in {source_name}")
        return {"total": 0, "success": 0, "failed": 0, "skipped": 0}
    
    print(f"  ✅ {len(all_tables)} tables found")
    
    # Apply filter
    tables = filter_tables(all_tables, table_filter)
    
    if table_filter:
        print(f"  🔍 After applying filter: {len(tables)} tables")
        if len(tables) < len(all_tables):
            excluded = len(all_tables) - len(tables)
            print(f"     ({excluded} tables excluded by filter)")
    
    if not tables:
        print(f"  ⚠️  No tables match the filter criteria")
        return {"total": 0, "success": 0, "failed": 0, "skipped": 0}
    
    # Create shortcuts
    print(f"\n  🔗 Creating shortcuts...")
    success_count = 0
    failed_count = 0
    skipped_count = 0
    failed_tables = []
    
    for i, table in enumerate(tables, 1):
        table_name = table["name"]
        
        # Skip if shortcut already exists (refresh mode)
        if refresh_mode and table_name in existing_shortcuts:
            print(f"     [{i:3d}/{len(tables)}] {table_name}... ⏭️  (already exists)")
            skipped_count += 1
            continue
        
        # Path in target lakehouse: Tables/schema_name
        shortcut_path = f"Tables/{schema_name}"
        
        # Path in source lakehouse: Tables/table_name
        source_path = f"Tables/{table_name}"
        
        print(f"     [{i:3d}/{len(tables)}] {table_name}...", end=" ")
        
        success, message = create_shortcut(
            target_workspace_id=target_workspace_id,
            target_lakehouse_id=target_lakehouse_id,
            shortcut_path=shortcut_path,
            shortcut_name=table_name,
            source_workspace_id=source_workspace_id,
            source_lakehouse_id=source_lakehouse_id,
            source_path=source_path
        )
        
        if success:
            print("✅")
            success_count += 1
        else:
            print(f"❌")
            print(f"          Error: {message}")
            failed_tables.append(table_name)
            failed_count += 1
        
        # Small pause to avoid rate limiting
        time.sleep(0.5)
    
    # Source summary
    print(f"\n  📊 Summary for {source_name}:")
    print(f"     ✅ Successful: {success_count}")
    if refresh_mode and skipped_count > 0:
        print(f"     ⏭️  Skipped (already exist): {skipped_count}")
    print(f"     ❌ Failed: {failed_count}")
    
    if failed_tables:
        print(f"\n  ⚠️  Tables that failed:")
        for table in failed_tables[:10]:  # Show maximum 10
            print(f"     - {table}")
        if len(failed_tables) > 10:
            print(f"     ... and {len(failed_tables) - 10} more")
    
    return {
        "total": len(tables),
        "success": success_count,
        "failed": failed_count,
        "skipped": skipped_count
    }

def setup_consolidated_lakehouse(target_workspace_id: str, new_lakehouse_name: str, 
                                sources_config: Dict) -> Optional[str]:
    """
    Creates a consolidated lakehouse with shortcuts organized by schema
    """
    print("="*70)
    print(f"Starting consolidated lakehouse setup: {new_lakehouse_name}")
    print("="*70)
    
    # 1. Create new lakehouse
    print(f"\n[STEP 1] Creating target lakehouse...")
    new_lakehouse_id = create_lakehouse(target_workspace_id, new_lakehouse_name)
    
    if not new_lakehouse_id:
        print("❌ Could not create lakehouse")
        return None
    
    print(f"✅ Lakehouse created successfully")
    print(f"   ID: {new_lakehouse_id}")
    print(f"   Workspace: {target_workspace_id}")
    
    # Wait for lakehouse to initialize completely
    print("\n⏳ Waiting for lakehouse initialization...")
    time.sleep(10)
    
    # Global statistics
    total_tables = 0
    total_success = 0
    total_failed = 0
    
    # 2. Process each source
    for source_name, source_config in sources_config.items():
        stats = process_source(
            target_workspace_id=target_workspace_id,
            target_lakehouse_id=new_lakehouse_id,
            source_name=source_name,
            source_config=source_config,
            refresh_mode=False
        )
        
        total_tables += stats["total"]
        total_success += stats["success"]
        total_failed += stats["failed"]
    
    # Final summary
    print(f"\n{'='*70}")
    print(f"FINAL SUMMARY")
    print(f"{'='*70}")
    print(f"  Lakehouse: {new_lakehouse_name}")
    print(f"  ID: {new_lakehouse_id}")
    print(f"  Workspace: {target_workspace_id}")
    print(f"\n  📊 Statistics:")
    print(f"     Total tables processed: {total_tables}")
    print(f"     ✅ Shortcuts created: {total_success}")
    print(f"     ❌ Shortcuts failed: {total_failed}")
    
    if total_success > 0:
        success_rate = (total_success / total_tables * 100) if total_tables > 0 else 0
        print(f"     📈 Success rate: {success_rate:.1f}%")
    
    print(f"{'='*70}\n")
    
    return new_lakehouse_id

def refresh_consolidated_lakehouse(target_workspace_id: str, target_lakehouse_id: str, 
                                   sources_config: Dict) -> Dict:
    """
    Refreshes shortcuts in an existing consolidated lakehouse
    Only creates shortcuts for new tables that don't exist yet
    """
    print("="*70)
    print(f"Refreshing consolidated lakehouse: {target_lakehouse_id}")
    print("="*70)
    
    # Verify target lakehouse exists
    print(f"\n🔍 Verifying target lakehouse...")
    if not verify_lakehouse(target_workspace_id, target_lakehouse_id):
        print("❌ Cannot access target lakehouse")
        return None
    print(f"✅ Target lakehouse accessible")
    
    # Global statistics
    total_tables = 0
    total_success = 0
    total_failed = 0
    total_skipped = 0
    
    # Process each source in refresh mode
    for source_name, source_config in sources_config.items():
        stats = process_source(
            target_workspace_id=target_workspace_id,
            target_lakehouse_id=target_lakehouse_id,
            source_name=source_name,
            source_config=source_config,
            refresh_mode=True
        )
        
        total_tables += stats["total"]
        total_success += stats["success"]
        total_failed += stats["failed"]
        total_skipped += stats["skipped"]
    
    # Final summary
    print(f"\n{'='*70}")
    print(f"REFRESH SUMMARY")
    print(f"{'='*70}")
    print(f"  Lakehouse ID: {target_lakehouse_id}")
    print(f"  Workspace: {target_workspace_id}")
    print(f"\n  📊 Statistics:")
    print(f"     Total tables found: {total_tables}")
    print(f"     ✅ New shortcuts created: {total_success}")
    print(f"     ⏭️  Shortcuts skipped (already exist): {total_skipped}")
    print(f"     ❌ Shortcuts failed: {total_failed}")
    
    print(f"{'='*70}\n")
    
    return {
        "total": total_tables,
        "success": total_success,
        "failed": total_failed,
        "skipped": total_skipped
    }

def add_source_to_lakehouse(target_workspace_id: str, target_lakehouse_id: str,
                           source_name: str, source_config: Dict) -> Dict:
    """
    Adds a new source to an existing consolidated lakehouse
    """
    print("="*70)
    print(f"Adding new source to lakehouse: {target_lakehouse_id}")
    print("="*70)
    
    # Verify target lakehouse exists
    print(f"\n🔍 Verifying target lakehouse...")
    if not verify_lakehouse(target_workspace_id, target_lakehouse_id):
        print("❌ Cannot access target lakehouse")
        return None
    print(f"✅ Target lakehouse accessible")
    
    # Process the new source
    stats = process_source(
        target_workspace_id=target_workspace_id,
        target_lakehouse_id=target_lakehouse_id,
        source_name=source_name,
        source_config=source_config,
        refresh_mode=False
    )
    
    # Summary
    print(f"\n{'='*70}")
    print(f"ADD SOURCE SUMMARY")
    print(f"{'='*70}")
    print(f"  Source: {source_name}")
    print(f"  Schema: {source_config['schema_name']}")
    print(f"  Lakehouse ID: {target_lakehouse_id}")
    print(f"\n  📊 Statistics:")
    print(f"     Tables processed: {stats['total']}")
    print(f"     ✅ Shortcuts created: {stats['success']}")
    print(f"     ❌ Shortcuts failed: {stats['failed']}")
    
    print(f"{'='*70}\n")
    
    return stats

StatementMeta(, 1bfba26f-2861-45c5-b30b-1a3ba2850f1c, 6, Finished, Available, Finished)

## Use cases

#### **Scenario 1: Initial Setup (First Time)**

In [ ]:
#### CONFIGURATION
target_workspace_id= target_workspace_id
new_lakehouse_name= "Consolidated_Lakehouse"

sources = {
    "bronze": {
        "workspace_id": "workspace-id-bronze",
        "lakehouse_id": "lakehouse-id-bronze",
        "schema_name": "bronze_schema",
        "table_filter": []  # Empty list = all tables, or specify: ["table1", "table2"]
    },
    "silver": {
        "workspace_id": "workspace-id-silver",
        "lakehouse_id": "lakehouse-id-silver", 
        "schema_name": "silver_schema",
        "table_filter": ["customer", "order"]  # Only these tables
    },
    "gold": {
        "workspace_id": "workspace-id-gold",
        "lakehouse_id": "lakehouse-id-gold",
        "schema_name": "gold_schema",
        "table_filter": []  # All tables
    }
}


In [ ]:
new_lakehouse_id = setup_consolidated_lakehouse(
    target_workspace_id=target_workspace_id,
    new_lakehouse_name="Consolidated_Lakehouse",
    sources_config=sources
)

#### **Scenario 2: Refresh (Add New Tables)**

In [ ]:
#### CONFIGURATION

target_workspace_id = target_workspace_id
target_lakehouse_id = "your-existing-lakehouse-id"


StatementMeta(, 1bfba26f-2861-45c5-b30b-1a3ba2850f1c, 10, Finished, Available, Finished)


OPTION 2: REFRESH LAKEHOUSE
Refreshing consolidated lakehouse: your-existing-lakehouse-id

🔍 Verifying target lakehouse...
❌ Cannot access target lakehouse


In [ ]:
refresh_consolidated_lakehouse(
    target_workspace_id=target_workspace_id,
    target_lakehouse_id=target_lakehouse_id,
    sources_config=sources
)

#### **Scenario 3: Add a New Source**


In [ ]:
#### CONFIGURATION
target_workspace_id = target_workspace_id
target_lakehouse_id = "your-existing-lakehouse-id"
source_name="schema_name_in_consolidated_lh",

# Define new source
new_source = {
    "workspace_id": "workspace-id-platinum",
    "lakehouse_id": "lakehouse-id-platinum",
    "schema_name": "platinum_schema",
    "table_filter": ["table_1", "table_2"]  # Or [] for all
    }

In [ ]:
add_source_to_lakehouse(
    target_workspace_id= target_workspace_id,
    target_lakehouse_id= target_lakehouse_id,
    source_name= source_name,
    source_config= new_source
)

#### **Scenario 4: Update Table Filters**

In [ ]:
#### CONFIGURATION

target_workspace_id = target_workspace_id
target_lakehouse_id = "your-existing-lakehouse-id"

updated_config = {
    "workspace_id": "workspace-id-platinum",
    "lakehouse_id": "lakehouse-id-platinum",
    "schema_name": "platinum_schema",
    "table_filter": ["table_1", "table_2", "table_3", "table_4"]  # Added 2 tables
}


In [ ]:
refresh_consolidated_lakehouse(
    target_workspace_id= target_workspace_id,
    target_lakehouse_id= target_lakehouse_id,
    source_name= source_name,
    sources_config={"silver": updated_silver}  # Only refresh silver
)

#### **Scenario 5: Daily refresh**

In [ ]:
def daily_refresh_job():
    """
    Scheduled job to refresh consolidated lakehouse
    Can be triggered by Fabric scheduling or orchestration
    """
    import datetime
    
    print(f"Starting scheduled refresh: {datetime.datetime.now()}")
    
    # Your standard configuration
    sources = {
        "bronze": {...},
        "silver": {...},
        "gold": {...}
    }
    
    # Refresh
    refresh_stats = refresh_consolidated_lakehouse(
        target_workspace_id=target_workspace_id,
        target_lakehouse_id="your-consolidated-lakehouse-id",
        sources_config=sources
    )
    
    # Log results
    if refresh_stats:
        print(f"✅ Refresh completed:")
        print(f"   New shortcuts: {refresh_stats['success']}")
        print(f"   Unchanged: {refresh_stats['skipped']}")
        print(f"   Failed: {refresh_stats['failed']}")
        
        # Alert if failures
        if refresh_stats['failed'] > 0:
            print("⚠️  WARNING: Some shortcuts failed - review logs")
    
    return refresh_stats

# Run the job
daily_refresh_job()

# Documentation

#### 1. Example Architecture
#### 2. Key Features
#### 3. Prerequisites
#### 4. Configuration
#### 5. Scenarios
#### 6. Good practice

### **1. Example Architecture**

```
Consolidated Lakehouse
├── bronze_schema/
│   ├── raw_customers (shortcut → Bronze LH)
│   ├── raw_orders (shortcut → Bronze LH)
│   └── raw_products (shortcut → Bronze LH)
├── silver_schema/
│   ├── cleaned_customers (shortcut → Silver LH)
│   ├── cleaned_orders (shortcut → Silver LH)
│   └── cleaned_products (shortcut → Silver LH)
└── gold_schema/
    ├── dim_customer (shortcut → Gold LH)
    ├── dim_product (shortcut → Gold LH)
    └── fact_sales (shortcut → Gold LH)
```

Benefits:
- ✅ Single point of access to all data layers
- ✅ No data duplication (shortcuts only)
- ✅ Organized by business domains/schemas
- ✅ Easy to maintain and refresh
- ✅ Source lakehouses can be in different workspaces


---


### **2. Key features**

#### 1. Initial Setup
Create a new consolidated lakehouse with shortcuts to all specified source lakehouses.

#### 2. Intelligent Refresh
Automatically detect and create shortcuts for new tables that have been added to source lakehouses since the last run. Existing shortcuts are skipped gracefully (no errors).

#### 3. Add New Sources
Dynamically add new data sources to an existing consolidated lakehouse without rebuilding everything.

#### 4. Table Filtering
Control exactly which tables are included from each source:
- Include all tables: `"table_filter": []`
- Include specific tables: `"table_filter": ["table1", "table2", "table3"]`

#### 5. Cross-Workspace Support
Source lakehouses can be located in different workspaces. The tool handles authentication and access automatically.

#### 6. Error Handling
- Gracefully handles existing shortcuts (no duplicate errors)
- Validates lakehouse access before processing
- Detailed error reporting for failed operations
- Continues processing even if individual tables fail

---

### **3. Prerequisites** 

#### Required Permissions
Your account needs:
- **Read permissions** on all source workspace(s) and lakehouse(s)
- **Write permissions** on the target workspace where the consolidated lakehouse will be created
- **Contributor or Admin role** is recommended

#### Required Libraries
The notebook uses:
- `requests` - HTTP library (pre-installed)
- `mssparkutils` - Fabric utilities (pre-installed)
- `json`, `time`, `typing` - Standard Python libraries

---


### **4. Configuration**

#### Step 1: Define Target Workspace
```python
# Workspace where the consolidated lakehouse will be created
target_workspace_id = "your-target-workspace-id"
```

**How to find Workspace ID:**
1. Navigate to your workspace in Fabric
2. Look at the URL: `https://app.fabric.microsoft.com/groups/{workspace-id}/...`
3. Copy the GUID between `/groups/` and the next `/`

#### Step 2: Configure Source Lakehouses
```python
sources = {
    "bronze": {
        "workspace_id": "workspace-id-where-bronze-lives",
        "lakehouse_id": "bronze-lakehouse-id",
        "schema_name": "bronze_schema",
        "table_filter": []  # Empty = all tables
    },
    "silver": {
        "workspace_id": "workspace-id-where-silver-lives",
        "lakehouse_id": "silver-lakehouse-id",
        "schema_name": "silver_schema",
        "table_filter": ["customers", "orders"]  # Only these tables
    },
    "gold": {
        "workspace_id": "workspace-id-where-gold-lives",
        "lakehouse_id": "gold-lakehouse-id",
        "schema_name": "gold_schema",
        "table_filter": []  # All tables
    }
}
```

**Configuration Parameters:**
- `workspace_id`: Workspace GUID where the source lakehouse is located
- `lakehouse_id`: Lakehouse GUID of the source
- `schema_name`: Name for the schema in the consolidated lakehouse (appears as folder)
- `table_filter`: List of table names to include (empty list = all tables)

**How to find Lakehouse ID:**
1. Open the lakehouse in Fabric
2. Look at the URL: `https://app.fabric.microsoft.com/groups/{workspace-id}/lakehouses/{lakehouse-id}`
3. Copy the GUID after `/lakehouses/`

---


### **5. Scenarios**

#### Scenario 1: Initial Setup (First Time)

**When to use:** You're creating a consolidated lakehouse for the first time.
```python
# Define your sources (see Configuration section)
sources = {
    "bronze": {...},
    "silver": {...},
    "gold": {...}
}

# Create the consolidated lakehouse
consolidated_lh_id = setup_consolidated_lakehouse(
    target_workspace_id=target_workspace_id,
    new_lakehouse_name="My_Consolidated_Lakehouse",
    sources_config=sources
)

print(f"✅ Consolidated Lakehouse created with ID: {consolidated_lh_id}")
```

**What happens:**
1. Creates a new lakehouse in the target workspace
2. Connects to each source lakehouse
3. Creates shortcuts for all tables (respecting filters)
4. Organizes shortcuts into schemas
5. Provides detailed statistics

**Expected Output:**
```
======================================================================
Starting consolidated lakehouse setup: My_Consolidated_Lakehouse
======================================================================

[STEP 1] Creating target lakehouse...
✅ Lakehouse created successfully
   ID: abc-123-def-456
   Workspace: xyz-789-workspace

======================================================================
[SOURCE] BRONZE
======================================================================
  Workspace ID: bronze-workspace-123
  Lakehouse ID: bronze-lh-456
  Target Schema: bronze_schema
  Table Filter: All tables

  🔍 Verifying access to source lakehouse...
  ✅ Lakehouse accessible

  📋 Getting table list...
  ✅ 25 tables found

  🔗 Creating shortcuts...
     [  1/25] customers... ✅
     [  2/25] orders... ✅
     ...

  📊 Summary for bronze:
     ✅ Successfully created: 25
     ❌ Failed: 0

[Processing continues for silver and gold...]

======================================================================
FINAL SUMMARY
======================================================================
  Lakehouse: My_Consolidated_Lakehouse
  ID: abc-123-def-456
  Workspace: xyz-789-workspace

  📊 Statistics:
     Total tables processed: 75
     ✅ Shortcuts created: 75
     ❌ Shortcuts failed: 0
     📈 Success rate: 100.0%
======================================================================
```

---

#### Scenario 2: Refresh (Add New Tables)

**When to use:** New tables have been added to your source lakehouses and you want to add them to the consolidated lakehouse.
```python
# Use the same sources configuration as initial setup
sources = {
    "bronze": {...},
    "silver": {...},
    "gold": {...}
}

# Refresh to catch new tables
refresh_stats = refresh_consolidated_lakehouse(
    target_workspace_id=target_workspace_id,
    target_lakehouse_id="your-existing-consolidated-lakehouse-id",
    sources_config=sources
)
```

**What happens:**
1. Connects to each source lakehouse
2. Gets current list of tables
3. Compares with existing shortcuts
4. Creates shortcuts ONLY for new tables
5. Skips existing shortcuts (no errors)

**Expected Output:**
```
======================================================================
Refreshing consolidated lakehouse: abc-123-def-456
======================================================================

🔍 Verifying target lakehouse...
✅ Target lakehouse accessible

======================================================================
[SOURCE] BRONZE
======================================================================
  Mode: REFRESH

  🔗 Creating shortcuts...
     [  1/28] customers... ⏭️  (already exists)
     [  2/28] orders... ⏭️  (already exists)
     [  3/28] new_table_1... ✅
     [  4/28] new_table_2... ✅
     ...

  📊 Summary for bronze:
     ✅ Successfully created: 3
     ⏭️  Already exist: 25
     ❌ Failed: 0

======================================================================
REFRESH SUMMARY
======================================================================
  📊 Statistics:
     Total tables found: 80
     ✅ New shortcuts created: 5
     ⏭️  Shortcuts already exist: 75
     ❌ Shortcuts failed: 0
======================================================================
```

**Use Cases:**
- Daily/weekly scheduled refresh to catch new tables
- After ETL processes add new tables to source lakehouses
- Maintenance after data pipeline updates

---

#### Scenario 3: Add a New Source

**When to use:** You want to add a completely new data source (e.g., adding a "Platinum" layer or external data) to your existing consolidated lakehouse.
```python
# Define the new source
platinum_source = {
    "workspace_id": "platinum-workspace-id",
    "lakehouse_id": "platinum-lakehouse-id",
    "schema_name": "platinum_schema",
    "table_filter": []  # All tables, or specify specific ones
}

# Add to existing consolidated lakehouse
add_stats = add_source_to_lakehouse(
    target_workspace_id=target_workspace_id,
    target_lakehouse_id="your-existing-consolidated-lakehouse-id",
    source_name="platinum",
    source_config=platinum_source
)
```

**What happens:**
1. Verifies access to target lakehouse
2. Connects to the new source lakehouse
3. Creates shortcuts for all tables from the new source
4. Organizes them in a new schema

**Expected Output:**
```
======================================================================
Adding new source to lakehouse: abc-123-def-456
======================================================================

🔍 Verifying target lakehouse...
✅ Target lakehouse accessible

======================================================================
[SOURCE] PLATINUM
======================================================================
  Workspace ID: platinum-workspace-789
  Lakehouse ID: platinum-lh-012
  Target Schema: platinum_schema
  Table Filter: All tables

  🔗 Creating shortcuts...
     [  1/15] advanced_analytics... ✅
     [  2/15] ml_features... ✅
     ...

======================================================================
ADD SOURCE SUMMARY
======================================================================
  Source: platinum
  Schema: platinum_schema
  
  📊 Statistics:
     Tables processed: 15
     ✅ Shortcuts created: 15
     ❌ Shortcuts failed: 0
======================================================================
```

---

#### Scenario 4: Update Table Filters

**When to use:** You want to add more tables from an existing source that was previously filtered.

**Example:** You originally only included `["customers", "orders"]` from Silver, but now you want to add `"products"` and `"inventory"`.
```python
# Original configuration
original_silver = {
    "workspace_id": "silver-workspace",
    "lakehouse_id": "silver-lh",
    "schema_name": "silver_schema",
    "table_filter": ["customers", "orders"]
}

# Updated configuration - add more tables
updated_silver = {
    "workspace_id": "silver-workspace",
    "lakehouse_id": "silver-lh",
    "schema_name": "silver_schema",
    "table_filter": ["customers", "orders", "products", "inventory"]  # Added 2 tables
}

# Refresh with updated configuration
refresh_stats = refresh_consolidated_lakehouse(
    target_workspace_id=target_workspace_id,
    target_lakehouse_id="your-consolidated-lakehouse-id",
    sources_config={"silver": updated_silver}  # Only refresh silver
)
```

**What happens:**
1. Processes the updated filter
2. Skips "customers" and "orders" (already exist)
3. Creates shortcuts for "products" and "inventory"

---

#### Scenario 5: Scheduled Maintenance

**When to use:** You want to run a regular job (daily/weekly) to keep your consolidated lakehouse up to date.
```python
def daily_refresh_job():
    """
    Scheduled job to refresh consolidated lakehouse
    Can be triggered by Fabric scheduling or orchestration
    """
    import datetime
    
    print(f"Starting scheduled refresh: {datetime.datetime.now()}")
    
    # Your standard configuration
    sources = {
        "bronze": {...},
        "silver": {...},
        "gold": {...}
    }
    
    # Refresh
    refresh_stats = refresh_consolidated_lakehouse(
        target_workspace_id=target_workspace_id,
        target_lakehouse_id="your-consolidated-lakehouse-id",
        sources_config=sources
    )
    
    # Log results
    if refresh_stats:
        print(f"✅ Refresh completed:")
        print(f"   New shortcuts: {refresh_stats['success']}")
        print(f"   Unchanged: {refresh_stats['skipped']}")
        print(f"   Failed: {refresh_stats['failed']}")
        
        # Alert if failures
        if refresh_stats['failed'] > 0:
            print("⚠️  WARNING: Some shortcuts failed - review logs")
    
    return refresh_stats

# Run the job
daily_refresh_job()
```

**Scheduling Options:**
1. **Fabric Pipeline:** Create a pipeline with a Notebook activity
2. **Cron Job:** If using external orchestration
3. **Manual:** Run on-demand when needed

---


### **6. Good practice**

#### Keep a record of your configuration:
```python
# Configuration Documentation
"""
Consolidated Lakehouse: Analytics_Hub
Created: 2026-01-15
Owner: Data Team
Purpose: Unified view of medallion architecture

Sources:
- Bronze: Raw data from all systems (120 tables)
- Silver: Cleaned and validated (85 tables, filtered)
- Gold: Business-ready aggregations (45 tables)

Refresh Schedule: Daily at 2 AM UTC
Last Updated: 2026-01-15
"""

sources = {
    # ... your configuration
}
```


# Troubleshoting

#### Common Issues and Solutions

#### **Issue 1: "Cannot access lakehouse"**
```
❌ Cannot access lakehouse bronze
   Check permissions and verify the ID is correct
```

**Solutions:**
1. Verify the lakehouse ID is correct
   - Open lakehouse in browser
   - Check URL for correct GUID
2. Check workspace permissions
   - You need at least Read access
3. Verify the lakehouse hasn't been deleted or renamed

**How to verify:**
```python
# Test lakehouse access manually
test_url = f"{base_url}/workspaces/{workspace_id}/lakehouses/{lakehouse_id}"
response = requests.get(test_url, headers=headers)
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")
```

---

#### **Issue 2: "No tables found"**
```
⚠️  No tables found in bronze
```

**Solutions:**
1. Check if source lakehouse actually has tables
   - Open lakehouse in Fabric UI
   - Verify tables exist in Tables section
2. Tables might be in Files section (not supported)
   - Shortcuts only work with Delta tables
3. Check if tables are still loading

---

#### **Issue 3: Filtered tables not found**
```
⚠️  No tables match the filter criteria
```

**Solutions:**
1. Check spelling of table names in filter
2. Table names are case-sensitive
3. Remove filter temporarily to see all available tables:
```python
   # Temporarily set filter to empty to see all tables
   "table_filter": []
```

---

#### **Issue 4: Some shortcuts fail**
```
❌ Shortcuts failed: 3
```

**Solutions:**
1. Check the detailed error messages in output
2. Common causes:
   - Source table was deleted
   - Permission changes
   - Network issues
3. Run refresh again - transient errors often resolve
4. Check specific table:
```python
   # Debug specific table
   tables = get_lakehouse_tables(workspace_id, lakehouse_id)
   print([t['name'] for t in tables])
```

---

#### **Issue 5: Rate limiting errors**
```
Error: 429 - Too Many Requests
```

**Solutions:**
1. The tool includes automatic delays (`time.sleep(0.5)`)
2. If still occurring, increase delay:
```python
   # In process_source function, increase sleep time
   time.sleep(1.0)  # Instead of 0.5
```
3. Process sources in smaller batches

---

#### Issue 6: **Authentication errors**
```
Error: 401 - Unauthorized
```

**Solutions:**
1. Token may have expired - rerun the notebook
2. Verify you're using the correct authentication:
```python
   token = mssparkutils.credentials.getToken("pbi")
```
3. Check if workspace/lakehouse access was revoked

---

### Debug Mode

Enable detailed logging for troubleshooting:
```python
# Add at the top of your notebook
DEBUG = True

# Modify functions to include debug output
if DEBUG:
    print(f"DEBUG: Attempting to create shortcut")
    print(f"  Target: {target_lakehouse_id}")
    print(f"  Source: {source_lakehouse_id}")
    print(f"  Table: {table_name}")


---
# Getting Help

**Useful Links:**
- [Microsoft Fabric Documentation](https://learn.microsoft.com/fabric/)
- [OneLake Shortcuts Documentation](https://learn.microsoft.com/fabric/onelake/onelake-shortcuts)
- [Fabric REST API Reference](https://learn.microsoft.com/rest/api/fabric/)

---

## Version History

| Version | Date | Changes |
|---------|------|---------|
| 1.0 | 2026-01-15 | Initial release |

---

## Support and Feedback

For questions, issues, or suggestions:
- Contact: eneko.egiguren.gomez@gmail.com
- Linkedin: https://www.linkedin.com/in/enekoegiguren/
---

**Remember:** This tool creates shortcuts, not copies. Changes in source lakehouses are immediately reflected in the consolidated lakehouse. There is no data duplication, which keeps storage costs low but means source lakehouses must remain accessible.